# Soft Prompting Pipeline Demo

This notebook demonstrates how to use the soft prompting pipeline to discover behavioral differences between models.

In [ ]:
import sys
sys.path.append('../')

import torch
from pathlib import Path
import yaml
import logging

from src.soft_prompting.core.pipeline import DivergencePipeline
from src.soft_prompting.config.configs import ExperimentConfig
from src.soft_prompting.models.model_manager import ModelPairManager
from src.soft_prompting.models.registry import ModelRegistry

## Setup Configuration

In [ ]:
# Load configuration
config_path = Path('../src/soft_prompting/config/divergence_experiments.yaml')
with open(config_path) as f:
    config_dict = yaml.safe_load(f)

# Setup experiment directory
output_dir = Path('outputs/demo')
output_dir.mkdir(parents=True, exist_ok=True)

# Create experiment config
config = ExperimentConfig.from_dict({
    "name": "demo_experiment",
    "output_dir": str(output_dir),
    "model_1_name": "HuggingFaceTB/SmolLM-135M-Instruct",  # Small model for testing
    "model_2_name": "HuggingFaceTB/SmolLM-135M",
    **config_dict
})

## Initialize Models

In [ ]:
# Setup model registry and manager
registry = ModelRegistry()
model_manager = ModelPairManager(
    registry=registry,
    device="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.float16,
    load_in_8bit=False
)

# Load model pair
model_1, model_2, tokenizer = model_manager.load_model_pair(
    config.model_1_name,
    use_cache=True
)

## Run Pipeline

In [ ]:
# Initialize pipeline
pipeline = DivergencePipeline(config=config)

# Run pipeline
try:
    results = pipeline.run()
    print("Pipeline completed successfully!")
except Exception as e:
    print(f"Error running pipeline: {e}")
finally:
    pipeline.cleanup()

## Analyze Results

In [ ]:
from src.soft_prompting.analysis.divergence_analyzer import DivergenceAnalyzer

# Initialize analyzer
analyzer = DivergenceAnalyzer(
    metrics=pipeline.trainer.metrics,
    output_dir=output_dir
)

# Generate analysis report
report = analyzer.generate_report(
    dataset=results['dataset'],
    output_file="analysis_report.json"
)

# Display key findings
print("\nKey Findings:")
print(f"Number of high-divergence examples: {report['divergence_patterns']['num_high_divergence']}")
print(f"Mean semantic difference: {report['divergence_patterns']['mean_semantic_diff']:.3f}")

print("\nBehavior Clusters:")
for cluster_id, stats in report['behavior_clusters'].items():
    print(f"\nCluster {cluster_id}:")
    print(f"Size: {stats['size']}")
    print(f"Mean divergence: {stats['mean_divergence']:.3f}")
    print("Example prompts:")
    for prompt in stats['example_prompts']:
        print(f"- {prompt}")